In [6]:
import pandas as pd

# Cargar el dataset que contiene las columnas necesarias
try:
    df = pd.read_csv("dataset_final_preparado.csv")
    print("Dataset cargado exitosamente.")
except FileNotFoundError:
    print("Error: dataset_final_preparado.csv no encontrado. Asegúrate de que el archivo existe.")
    exit()

print(f"Filas originales: {len(df)}")

# Eliminar filas con valores nulos o vacíos en 'Ciudad' y 'Comentario'
# Keep original df for potential future use before dropping columns
df_cleaned = df.dropna(subset=['Ciudad', 'Comentario']).copy()
df_cleaned = df_cleaned[df_cleaned['Ciudad'].astype(str).str.strip() != '']
df_cleaned = df_cleaned[df_cleaned['Comentario'].astype(str).str.strip() != '']

print(f"Filas después de eliminar nulos/vacíos en 'Ciudad' o 'Comentario': {len(df_cleaned)}")

# Eliminar la columna 'Comentario'
if 'Comentario' in df_cleaned.columns:
    df_cleaned = df_cleaned.drop(columns=['Comentario']).copy()
    print("Columna 'Comentario' eliminada.")

# Now, instead of selecting a subset, keep all remaining columns
df_procesado = df_cleaned.copy()

print("\nDataset después de la limpieza:")
display(df_procesado.head())

print(f"\nFilas finales: {len(df_procesado)}")

# Puedes guardar este DataFrame procesado si lo necesitas para pasos posteriores
df_procesado.to_csv("dataset_procesado_para_prediccion.csv", index=False)
print("\nDataset procesado guardado como dataset_procesado_para_prediccion.csv")

Dataset cargado exitosamente.
Filas originales: 9236
Filas después de eliminar nulos/vacíos en 'Ciudad' o 'Comentario': 9236
Columna 'Comentario' eliminada.

Dataset después de la limpieza:


,Ciudad,Categoría del problema,Categoria numerica,Nivel de urgencia,Fecha del reporte
0,Manizales,Seguridad,3,3,2023-08-11
1,Santa Marta,Educación,2,1,2023-04-12
2,Medellín,Medio Ambiente,4,1,2023-06-09
3,Bogotá,Medio Ambiente,4,1,2023-06-04
4,Manizales,Educación,2,4,2024-07-10



Filas finales: 9236

Dataset procesado guardado como dataset_procesado_para_prediccion.csv


In [7]:
# Paso 1: Cargar librerías
import pandas as pd

# Paso 2: Subir y cargar el dataset



df = pd.read_csv('dataset_procesado_para_prediccion.csv')

# Paso 3: Convertir fecha y crear mes
df['Fecha'] = pd.to_datetime(df['Fecha del reporte'], format='%Y %m %d', errors='coerce')
df = df.dropna(subset=['Fecha', 'Ciudad', 'Categoría del problema', 'Nivel de Urgencia'])
df['Mes'] = df['Fecha'].dt.to_period('M')

# Paso 4: Calcular gravedad mensual por ciudad y categoría
gravedad_mensual = df.groupby(['Ciudad', 'Categoría del problema', 'Mes']).agg(
    Gravedad=('Nivel de Urgencia', 'sum'),
    Urgencia_Promedio=('Nivel de Urgencia', 'mean'),
    Reportes=('Nivel de Urgencia', 'count')
).reset_index()

# Convertir Mes a timestamp para operar
gravedad_mensual['Mes_dt'] = gravedad_mensual['Mes'].dt.start_time

# Paso 5: Definir "reciente" = últimos 3 meses completos
todos_los_meses = sorted(gravedad_mensual['Mes_dt'].unique())
if len(todos_los_meses) < 4:
    print("⚠️ Menos de 4 meses de datos. Usando todos los datos disponibles.")
    meses_recientes = todos_los_meses
else:
    meses_recientes = todos_los_meses[-3:]

# Separar datos recientes e históricos
reciente = gravedad_mensual[gravedad_mensual['Mes_dt'].isin(meses_recientes)]
historico = gravedad_mensual[~gravedad_mensual['Mes_dt'].isin(meses_recientes)]

# Paso 6: Agregar estadísticas
hist_stats = historico.groupby(['Ciudad', 'Categoría del problema'])['Gravedad'].mean().rename('Gravedad_Hist_Mean').reset_index()
rec_stats = reciente.groupby(['Ciudad', 'Categoría del problema'])['Gravedad'].mean().rename('Gravedad_Rec_Mean').reset_index()
urg_rec = reciente.groupby(['Ciudad', 'Categoría del problema'])['Urgencia_Promedio'].mean().rename('Urgencia_Rec_Prom').reset_index()

# Combinar
alertas = hist_stats.merge(rec_stats, on=['Ciudad', 'Categoría del problema'], how='outer').fillna(0)
alertas = alertas.merge(urg_rec, on=['Ciudad', 'Categoría del problema'], how='left').fillna(0)

# Calcular crecimiento (%)
alertas['Crecimiento (%)'] = ((alertas['Gravedad_Rec_Mean'] - alertas['Gravedad_Hist_Mean']) /
                              (alertas['Gravedad_Hist_Mean'] + 1e-5)) * 100

# Paso 7: Filtrar alertas críticas
alertas_criticas = alertas[
    (alertas['Crecimiento (%)'] > 25) &           # Aumento >25%
    (alertas['Urgencia_Rec_Prom'] >= 3.0) &      # Urgencia alta
    (alertas['Gravedad_Rec_Mean'] > 2)            # Al menos algo de actividad reciente
].sort_values('Crecimiento (%)', ascending=False)

# Paso 8: Mostrar resultados
print("🚨 ALERTAS DE PROBLEMAS EMERGENTES (últimos 3 meses):")
if not alertas_criticas.empty:
    print(alertas_criticas[['Ciudad', 'Categoría del problema', 'Crecimiento (%)', 'Urgencia_Rec_Prom']].to_string(index=False))
else:
    print("ℹ️ No hay alertas críticas. Mostrando top 10 con mayor crecimiento:")
    top10 = alertas.sort_values('Crecimiento (%)', ascending=False).head(10)
    print(top10[['Ciudad', 'Categoría del problema', 'Crecimiento (%)', 'Urgencia_Rec_Prom']].to_string(index=False))

# Guardar resultados
alertas.to_csv('alertas_tendencias_ciudad_categoria.csv', index=False)
print("\n✅ Resultados guardados en 'alertas_tendencias_ciudad_categoria.csv'")

KeyError: ['Nivel de Urgencia']

In [ ]:
# Paso 1: Instalar librerías
%pip install xgboost pandas scikit-learn xgboost --quiet

# Paso 2: Importar
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from datetime import timedelta
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import roc_auc_score

# Paso 3: Cargar el dataset con comentarios
df = pd.read_csv('dataset_de_prediccion.csv', header=None)
df.columns = ['Ciudad', 'Comentario', 'Categoría', 'Col4', 'Nivel de Urgencia', 'Fecha']

# Limpiar y convertir
df['Nivel de Urgencia'] = pd.to_numeric(df['Nivel de Urgencia'], errors='coerce')
df = df.dropna(subset=['Nivel de Urgencia'])
df['Nivel de Urgencia'] = df['Nivel de Urgencia'].astype(int)

df['Fecha'] = pd.to_datetime(df['Fecha'], format='%Y %m %d', errors='coerce')
df = df.dropna(subset=['Fecha', 'Ciudad', 'Categoría', 'Comentario'])
df = df.sort_values('Fecha').reset_index(drop=True)

print(f"✅ Datos cargados: {len(df)} registros desde {df['Fecha'].min()} hasta {df['Fecha'].max()}")

# Paso 4: Crear etiqueta: crisis = al menos un reporte con urgencia ≥3 en los próximos 30 días
def crear_etiqueta_grupo(grupo):
    grupo = grupo.sort_values('Fecha')
    fechas = grupo['Fecha'].values
    urgencias = grupo['Nivel de Urgencia'].values
    etiquetas = []
    for fecha_actual in fechas:
        ventana = (fechas > fecha_actual) & (fechas <= fecha_actual + np.timedelta64(30, 'D'))
        crisis = np.any(urgencias[ventana] >= 3)
        etiquetas.append(int(crisis))
    grupo['Crisis_30d'] = etiquetas
    return grupo

df = df.groupby(['Ciudad', 'Categoría']).apply(crear_etiqueta_grupo).reset_index(drop=True)

# Paso 5: Crear features avanzados
def crear_features_grupo(grupo):
    grupo = grupo.sort_values('Fecha')
    features = []
    for idx, row in grupo.iterrows():
        fecha = row['Fecha']
        hist = grupo[(grupo['Fecha'] < fecha) & (grupo['Fecha'] >= fecha - timedelta(days=30))]
        if len(hist) == 0:
            feat = {
                'reportes_7d': 0, 'reportes_15d': 0, 'reportes_30d': 0,
                'urgencia_prom_30d': 0, 'dias_ultimo_grave': 30,
                'tendencia_15d': 0,
                'mes_sin': np.sin(2 * np.pi * fecha.month / 12),
                'mes_cos': np.cos(2 * np.pi * fecha.month / 12)
            }
        else:
            r7 = hist[hist['Fecha'] >= fecha - timedelta(days=7)].shape[0]
            r15 = hist[hist['Fecha'] >= fecha - timedelta(days=15)].shape[0]
            r30 = len(hist)
            urg_prom = hist['Nivel de Urgencia'].mean()
            graves = hist[hist['Nivel de Urgencia'] >= 3]
            dias_ultimo = (fecha - graves['Fecha'].max()).days if len(graves) > 0 else 30

            hist_15 = hist[hist['Fecha'] >= fecha - timedelta(days=15)]
            tendencia = np.polyfit(np.arange(len(hist_15)), hist_15['Nivel de Urgencia'].values, 1)[0] if len(hist_15) > 2 else 0

            feat = {
                'reportes_7d': r7, 'reportes_15d': r15, 'reportes_30d': r30,
                'urgencia_prom_30d': urg_prom, 'dias_ultimo_grave': dias_ultimo,
                'tendencia_15d': tendencia,
                'mes_sin': np.sin(2 * np.pi * fecha.month / 12),
                'mes_cos': np.cos(2 * np.pi * fecha.month / 12)
            }
        features.append(feat)
    return pd.DataFrame(features, index=grupo.index)

# Aplicar features
feature_dfs = []
for (ciudad, cat), g in df.groupby(['Ciudad', 'Categoría']):
    feats = crear_features_grupo(g)
    feats['Ciudad'] = ciudad
    feats['Categoría'] = cat
    feature_dfs.append(feats)

X_feat = pd.concat(feature_dfs).sort_index()
df_final = pd.concat([df, X_feat], axis=1)

# Paso 6: Entrenar modelo
feature_cols = ['reportes_7d', 'reportes_15d', 'reportes_30d', 'urgencia_prom_30d',
                'dias_ultimo_grave', 'tendencia_15d', 'mes_sin', 'mes_cos']

X = df_final[feature_cols]
y = df_final['Crisis_30d']
X = X[~y.isna()]
y = y.dropna()

if len(X) < 50:
    print("⚠️ Pocos datos.")
else:
    # Validación temporal
    tscv = TimeSeriesSplit(n_splits=3)
    scores = []
    for tr, te in tscv.split(X):
        X_tr, X_te = X.iloc[tr], X.iloc[te]
        y_tr, y_te = y.iloc[tr], y.iloc[te]
        model = XGBClassifier(n_estimators=100, max_depth=4, random_state=42, eval_metric='logloss')
        model.fit(X_tr, y_tr)
        y_pred = model.predict_proba(X_te)[:, 1]
        if len(np.unique(y_te)) > 1:
            scores.append(roc_auc_score(y_te, y_pred))

    if scores:
        print(f"📊 AUC promedio: {np.mean(scores):.2f}")

    # Modelo final
    model_final = XGBClassifier(n_estimators=100, max_depth=4, random_state=42)
    model_final.fit(X, y)

    # Preparar datos limpios
    df_clean = df_final.reset_index(drop=True)
    df_clean = df_clean.loc[:, ~df_clean.columns.duplicated()]
    ultimos = df_clean.groupby(['Ciudad', 'Categoría']).tail(1).copy()

    # Predecir
    X_ult = ultimos[feature_cols]
    ultimos['Prob_Crisis_30d'] = model_final.predict_proba(X_ult)[:, 1]

    # Umbral dinámico (percentil 90)
    umbral = np.percentile(ultimos['Prob_Crisis_30d'], 90)
    alertas = ultimos[ultimos['Prob_Crisis_30d'] >= umbral].copy()

    # Recomendaciones por palabras clave
    def generar_recomendacion(comentario, categoria):
        if pd.isna(comentario):
            return "Requiere revisión por especialistas"
        c = comentario.lower()
        if "basura" in c or "recoleccion" in c or "recolect" in c:
            return "Aumentar frecuencia de recolección de basura"
        elif "agua potable" in c or "falta agua" in c:
            return "Verificar suministro de agua potable"
        elif "médicos" in c or "centro de salud" in c:
            return "Movilizar brigadas médicas o verificar disponibilidad"
        elif "oscuras" in c or "alumbrado" in c:
            return "Reparar o instalar alumbrado público"
        elif "internet" in c or "biblioteca" in c or "centros culturales" in c:
            return "Activar programas de acceso digital o bibliotecas móviles"
        elif "escuelas" in c or "educación" in c:
            return "Revisar infraestructura educativa"
        elif "policía" in c or "presencia policial" in c:
            return "Aumentar patrullaje en zonas críticas"
        elif "contaminacion" in c or "río" in c:
            return "Monitorear fuentes de contaminación hídrica"
        else:
            return "Requiere revisión por especialistas"

    alertas['Recomendación'] = alertas.apply(
        lambda row: generar_recomendacion(row['Comentario'], row['Categoría']), axis=1
    )

    # Mostrar y guardar
    print(f"\n🚨 ALERTAS (probabilidad ≥ {umbral:.2%}):")
    salida = alertas[['Ciudad', 'Categoría', 'Prob_Crisis_30d', 'Comentario', 'Recomendación']]
    print(salida.to_string(index=False))

    salida.to_csv('alertas_con_recomendaciones_especificas.csv', index=False)
    print("\n✅ Resultados guardados en 'alertas_con_recomendaciones_especificas.csv'")

Note: you may need to restart the kernel to use updated packages.


FileNotFoundError: [Errno 2] No such file or directory: 'dataset_de_prediccion.csv'

In [ ]:
# Paso 1: Cargar librerías
import pandas as pd

# Paso 2: Cargar datos
df = pd.read_csv('dataset_final_limpio_ciudades.csv', header=None)
df.columns = ['Ciudad', 'Comentario', 'Categoría', 'Col4', 'Nivel de Urgencia', 'Fecha']

# Limpiar
df['Nivel de Urgencia'] = pd.to_numeric(df['Nivel de Urgencia'], errors='coerce')
df = df.dropna(subset=['Ciudad', 'Comentario', 'Categoría', 'Nivel de Urgencia'])
df['Fecha'] = pd.to_datetime(df['Fecha'], format='%Y %m %d', errors='coerce')

# Paso 3: Función de recomendación
def generar_recomendacion(comentario, categoria):
    c = str(comentario).lower()
    if "basura" in c or "recoleccion" in c or "recolect" in c:
        return "Aumentar frecuencia de recolección de basura"
    elif "agua potable" in c or "falta agua" in c:
        return "Verificar suministro de agua potable"
    elif "médicos" in c or "centro de salud" in c:
        return "Movilizar brigadas médicas o verificar disponibilidad"
    elif "oscuras" in c or "alumbrado" in c:
        return "Reparar o instalar alumbrado público"
    elif "internet" in c or "biblioteca" in c or "centros culturales" in c:
        return "Activar programas de acceso digital o bibliotecas móviles"
    elif "escuelas" in c or "educación" in c:
        return "Revisar infraestructura educativa"
    elif "policía" in c or "presencia policial" in c:
        return "Aumentar patrullaje en zonas críticas"
    elif "contaminacion" in c or "río" in c:
        return "Monitorear fuentes de contaminación hídrica"
    else:
        return "Requiere revisión por especialistas"

# Paso 4: Para cada ciudad y categoría, tomar el comentario más grave reciente
df['Recomendación'] = df.apply(lambda row: generar_recomendacion(row['Comentario'], row['Categoría']), axis=1)

# Ordenar por fecha y urgencia
df = df.sort_values(['Ciudad', 'Categoría', 'Nivel de Urgencia', 'Fecha'], ascending=[True, True, False, False])

# Tomar el mejor comentario por ciudad+categoría
informe = df.groupby(['Ciudad', 'Categoría']).first().reset_index()

# Seleccionar columnas útiles
resultado = informe[['Ciudad', 'Categoría', 'Comentario', 'Nivel de Urgencia', 'Recomendación']]

# Paso 5: Mostrar y guardar
print("📋 Informe completo: Problemáticas y soluciones por ciudad")
print(resultado.to_string(index=False))

resultado.to_csv('informe_ciudades_problematicas_y_soluciones.csv', index=False)
print("\n✅ Informe guardado en 'informe_ciudades_problematicas_y_soluciones.csv'")

📋 Informe completo: Problemáticas y soluciones por ciudad
      Ciudad      Categoría                                   Comentario  Nivel de Urgencia                                             Recomendación
barranquilla      Educación         no hay suficientes escuelas publicas                4.0                         Revisar infraestructura educativa
barranquilla Medio Ambiente     la contaminacion del rio esta aumentando                4.0               Monitorear fuentes de contaminación hídrica
barranquilla          Salud         faltan medicos en el centro de salud                4.0     Movilizar brigadas médicas o verificar disponibilidad
barranquilla      Seguridad              queremos mas presencia policial                4.0                     Aumentar patrullaje en zonas críticas
      bogota      Educación         no hay suficientes escuelas publicas                4.0                         Revisar infraestructura educativa
      bogota Medio Ambiente   hay problema

In [ ]:
# Convertir alertas a JSON
alertas_json = alertas[[
    'Ciudad', 'Categoría', 'Prob_Crisis_30d', 'Comentario', 'Recomendación'
]].to_dict(orient='records')

# Guardar
import json
with open('alertas.json', 'w', encoding='utf-8') as f:
    json.dump(alertas_json, f, ensure_ascii=False, indent=2)

# Descargar
from google.colab import files
files.download('alertas.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Cargar datos
df = pd.read_csv('dataset_final_limpio_ciudades.csv', header=None)
df.columns = ['Ciudad', 'Comentario', 'Categoría', 'Col4', 'Nivel de Urgencia', 'Fecha']
df['Fecha'] = pd.to_datetime(df['Fecha'], format='%Y %m %d', errors='coerce')
df = df.dropna(subset=['Ciudad', 'Comentario', 'Categoría', 'Nivel de Urgencia'])

# Función de recomendación (misma que en landing)
def generar_recomendacion(comentario, categoria):
    c = str(comentario).lower()
    if "basura" in c or "recoleccion" in c or "recolect" in c:
        return "Aumentar frecuencia de recolección de basura"
    elif "agua potable" in c or "falta agua" in c:
        return "Verificar suministro de agua potable"
    elif "médicos" in c or "centro de salud" in c:
        return "Movilizar brigadas médicas o verificar disponibilidad"
    elif "oscuras" in c or "alumbrado" in c:
        return "Reparar o instalar alumbrado público"
    elif "internet" in c or "biblioteca" in c or "centros culturales" in c:
        return "Activar programas de acceso digital o bibliotecas móviles"
    elif "escuelas" in c or "educación" in c:
        return "Revisar infraestructura educativa"
    elif "policía" in c or "presencia policial" in c:
        return "Aumentar patrullaje en zonas críticas"
    elif "contaminacion" in c or "río" in c:
        return "Monitorear fuentes de contaminación hídrica"
    else:
        return "Requiere revisión por especialistas"

df['Recomendación'] = df.apply(lambda row: generar_recomendacion(row['Comentario'], row['Categoría']), axis=1)

# Ordenar: mayor urgencia primero, luego más reciente
df = df.sort_values(['Ciudad', 'Nivel de Urgencia', 'Fecha'], ascending=[True, False, False])

# Tomar el primer registro por ciudad (el más grave y reciente)
problemas_principales = df.groupby('Ciudad').first().reset_index()

# Seleccionar columnas útiles
resultado = problemas_principales[[
    'Ciudad', 'Categoría', 'Nivel de Urgencia', 'Comentario', 'Recomendación'
]]

# Guardar como JSON
import json
with open('problemas_principales_por_ciudad.json', 'w', encoding='utf-8') as f:
    json.dump(resultado.to_dict(orient='records'), f, ensure_ascii=False, indent=2)

# Descargar
from google.colab import files
files.download('problemas_principales_por_ciudad.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>